In [78]:
#I TRRIED WITH FIXING THE OUTLIERS BUT ALSO HERE AS THE CLUSTERING THE 
#SITUATION WORSE FIXING OUTLIERS AS MEDIAN. ANYWAY I THINK WE HAVE AN
#INSUFFICIENT NUMBER OF RULES. MAYBE THERE IS A PROBLEM SOMEWHERE
#I THINK WE HAVE TO BALANCE ISBADBUY BUT I DON'T KNOW HOW

%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [79]:
df = pd.read_csv('training.csv')

In [80]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [81]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [82]:
#df_out = df[['MMR_factor','VehBCost','VehOdo','WarrantyCost']]

In [83]:
#Q1 =  df_out.quantile(0.25)
#Q3 = df_out.quantile(0.75)
#IQR = Q3 - Q1

In [84]:
#df_in = df_out[~((df_out < (Q1 - 1.5 * IQR)) |(df_out > (Q3 + 1.5 * IQR))).any(axis=1)]

#df_in.shape

(56634, 4)

In [85]:
#BINNING NUMERICAL VALUES
#We have to use the sturges' rule but that gives us 17 bin but i think 10 is better because we obtain more rules
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False) 
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [104]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Trim','Color','WheelType','Size','VNST','IsBadBuy',
             'VehBCost_bin','VehOdo_bin','WarrantyCost_bin']]

In [105]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                0
VehicleAge             0
Make                   0
Trim                1911
Color                  7
WheelType           2577
Size                   4
VNST                   0
IsBadBuy               0
VehBCost_bin           0
VehOdo_bin             0
WarrantyCost_bin       0
dtype: int64

In [106]:
df_new.dropna(subset = ['Color','Size'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [107]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

In [108]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [109]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [110]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [111]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [112]:
df_new.isnull().sum()

Auction             0
VehicleAge          0
Make                0
Trim                0
Color               0
WheelType           0
Size                0
VNST                0
IsBadBuy            0
VehBCost_bin        0
VehOdo_bin          0
WarrantyCost_bin    0
dtype: int64

In [113]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Size,VNST,IsBadBuy,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA,2,KIA,EX,BLUE,Covers,MEDIUM,FL,1,"[3649.4, 7297.8)","[60271.0, 71360.2)","[462.0, 1165.6)"
1,MANHEIM,3,DODGE,SXT,SILVER,Covers,MEDIUM,FL,0,"[3649.4, 7297.8)","[82449.4, 93538.6)","[1165.6, 1869.2)"
2,OTHER,2,DODGE,SE,BLACK,Covers,MEDIUM,OK,0,"[7297.8, 10946.2)","[38092.6, 49181.8)","[462.0, 1165.6)"
3,OTHER,4,FORD,S,BROWN,Covers,VAN,AZ,0,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"
4,MANHEIM,4,CHRYSLER,Bas,BLUE,Alloy,VAN,TX,0,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"


In [114]:
#this code is necessary because if we don't add a string to the algorithm value..
#..when we run the algorithm it doesn't find the attribute values
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [115]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,WheelType,Size,VNST,IsBadBuy,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,EX_Trim,BLUE_Color,Covers_WheelType,MEDIUM_Size,FL_VNST,1_IsBadBuy,"[3649.4, 7297.8)_VehBCost_bin","[60271.0, 71360.2)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,SXT_Trim,SILVER_Color,Covers_WheelType,MEDIUM_Size,FL_VNST,0_IsBadBuy,"[3649.4, 7297.8)_VehBCost_bin","[82449.4, 93538.6)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,SE_Trim,BLACK_Color,Covers_WheelType,MEDIUM_Size,OK_VNST,0_IsBadBuy,"[7297.8, 10946.2)_VehBCost_bin","[38092.6, 49181.8)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,S_Trim,BROWN_Color,Covers_WheelType,VAN_Size,AZ_VNST,0_IsBadBuy,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,Bas_Trim,BLUE_Color,Alloy_WheelType,VAN_Size,TX_VNST,0_IsBadBuy,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"


In [116]:
#we have to pass the values of df_new as a list to run in the right way apriori algorithm
baskets = df_new.values.tolist()

In [117]:
#BASKET EXAMPLE
baskets[0]

['ADESA_Auction',
 '2_VehicleAge',
 'KIA_Make',
 'EX_Trim',
 'BLUE_Color',
 'Covers_WheelType',
 'MEDIUM_Size',
 'FL_VNST',
 '1_IsBadBuy',
 '[3649.4, 7297.8)_VehBCost_bin',
 '[60271.0, 71360.2)_VehOdo_bin',
 '[462.0, 1165.6)_WarrantyCost_bin']

In [118]:
from fim import apriori

In [119]:
#This is the explaination of the parameters of apriori
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [130]:
rules = apriori(baskets, supp=5, zmin=2, target='r', conf=80, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

662
0 -- ('[3649.4, 7297.8)_VehBCost_bin', ('SXT_Trim', 'DODGE_Make'), 59.311349036402575, 5.238543897216274, 1.405933881814368, 83.38783518639634)
1 -- ('0_IsBadBuy', ('SXT_Trim', 'DODGE_Make'), 87.65396145610278, 5.238543897216274, 1.030420866274744, 90.32047089601046)
2 -- ('DODGE_Make', ('SXT_Trim',), 17.733618843683082, 5.238543897216274, 5.639006955177743, 100.0)
3 -- ('[3649.4, 7297.8)_VehBCost_bin', ('SXT_Trim',), 59.311349036402575, 5.238543897216274, 1.405933881814368, 83.38783518639634)
4 -- ('0_IsBadBuy', ('SXT_Trim',), 87.65396145610278, 5.238543897216274, 1.030420866274744, 90.32047089601046)
5 -- ('0_IsBadBuy', ('SC_VNST',), 87.65396145610278, 5.814132762312634, 0.9879073113702104, 86.59398939304656)
6 -- ('0_IsBadBuy', ('PONTIAC_Make',), 87.65396145610278, 5.8586723768736615, 0.9994110674068953, 87.60233918128655)
7 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('[38092.6, 49181.8)_VehOdo_bin', 'Covers_WheelType'), 50.06423982869379, 5.027837259100643, 1.9110030122281598, 95.

In [131]:
#RULES EXAMPLE
#INDEX: 
# 0 : 'AMERICAN_Nationality'
# 1 : ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')
# 2 : 83.58515777572207 refers to 'Y' in report='YXlC'  that is relative head(head is 'AMERICAN_Nationality') item support as a percentage
# 3 : 20.27957652379484 refers to 'X' in report='YXlC'  that is X relative body(body is ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')) set  support as a percentage
# 4 : 1.0436783505509324 refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
# 5 : 87.23601959790506 refers to 'C' in report='YXlC'  that is rule confidence as a percentage
rules[4]

('0_IsBadBuy',
 ('SXT_Trim',),
 87.65396145610278,
 5.238543897216274,
 1.030420866274744,
 90.32047089601046)

In [132]:
#Code that filters all the association rules with a certain lift under
count = 0
for r in rules:
    if r[4]>1.3: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
        print('%s -- %s' % (str(count), r))
        count = count + 1
print('Rules with high lift: {}'.format(count))

0 -- ('[3649.4, 7297.8)_VehBCost_bin', ('SXT_Trim', 'DODGE_Make'), 59.311349036402575, 5.238543897216274, 1.405933881814368, 83.38783518639634)
1 -- ('DODGE_Make', ('SXT_Trim',), 17.733618843683082, 5.238543897216274, 5.639006955177743, 100.0)
2 -- ('[3649.4, 7297.8)_VehBCost_bin', ('SXT_Trim',), 59.311349036402575, 5.238543897216274, 1.405933881814368, 83.38783518639634)
3 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('[38092.6, 49181.8)_VehOdo_bin', 'Covers_WheelType'), 50.06423982869379, 5.027837259100643, 1.9110030122281598, 95.67291311754686)
4 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('[38092.6, 49181.8)_VehOdo_bin', '0_IsBadBuy'), 50.06423982869379, 6.348608137044969, 1.855144848688253, 92.87641662169455)
5 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('[38092.6, 49181.8)_VehOdo_bin',), 50.06423982869379, 6.912205567451821, 1.8538758913399966, 92.81288723667906)
6 -- ('[3649.4, 7297.8)_VehBCost_bin', ('VAN_Size', 'Covers_WheelType'), 59.311349036402575, 5.185438972162741, 1.416990321373874

In [133]:
#TASKS ON KAGGLE:
# 1) Frequent patterns extraction with different values of support and different types (i.e. frequent, close, maximal), (6 points)
# 2) Discussion of the most interesting frequent patterns and analyze how changes the number of patterns w.r.t. the min_sup parameter (7 points)

In [134]:
# s/a   sets/all   all     frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='a') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 945
Min support = 10 
 Itemsets = 214
Min support = 15 
 Itemsets = 80
Min support = 20 
 Itemsets = 43
Min support = 25 
 Itemsets = 26
Min support = 30 
 Itemsets = 14
Min support = 35 
 Itemsets = 6
Min support = 40 
 Itemsets = 5
Min support = 45 
 Itemsets = 2
Min support = 50 
 Itemsets = 1
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [135]:
# c     closed     closed  frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='c')
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 945
Min support = 10 
 Itemsets = 214
Min support = 15 
 Itemsets = 80
Min support = 20 
 Itemsets = 43
Min support = 25 
 Itemsets = 26
Min support = 30 
 Itemsets = 14
Min support = 35 
 Itemsets = 6
Min support = 40 
 Itemsets = 5
Min support = 45 
 Itemsets = 2
Min support = 50 
 Itemsets = 1
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [136]:
#m     maximal    maximal frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='m') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 347
Min support = 10 
 Itemsets = 94
Min support = 15 
 Itemsets = 36
Min support = 20 
 Itemsets = 24
Min support = 25 
 Itemsets = 14
Min support = 30 
 Itemsets = 14
Min support = 35 
 Itemsets = 6
Min support = 40 
 Itemsets = 5
Min support = 45 
 Itemsets = 2
Min support = 50 
 Itemsets = 1
Min support = 55 
 Itemsets = 0
Min support = 60 
 Itemsets = 0
Min support = 65 
 Itemsets = 0
Min support = 70 
 Itemsets = 0
Min support = 75 
 Itemsets = 0
Min support = 80 
 Itemsets = 0
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [137]:
#TASK ON KAGGLE:
# 3) Association rules extraction with different values of confidence (6 points)
# 4) Discussion of the most interesting rules and analyze how changes the number of rules w.r.t. the min_conf parameter, histogram of rules' confidence and lift (7 points)

In [138]:
#EXTRACT RULES
for i in range(5,101,5):
    for j in range(5,101,5):
        rules = apriori(baskets, supp=i, zmin=2, target='r', conf=j, report='YXlC') 
        print("Support = {} \nConfidence = {} \nRules = {}".format(i,j,len(rules)))
        #Code that filters all the association rules with a certain lift under
        count = 0
        for r in rules:
            if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
                count = count + 1
        print('Rules with high lift: {}'.format(count))
        print('\n')

Support = 5 
Confidence = 5 
Rules = 41141
Rules with high lift: 7617


Support = 5 
Confidence = 10 
Rules = 28290
Rules with high lift: 4520


Support = 5 
Confidence = 15 
Rules = 20054
Rules with high lift: 3581


Support = 5 
Confidence = 20 
Rules = 13701
Rules with high lift: 2663


Support = 5 
Confidence = 25 
Rules = 8933
Rules with high lift: 1787


Support = 5 
Confidence = 30 
Rules = 6553
Rules with high lift: 1287


Support = 5 
Confidence = 35 
Rules = 5323
Rules with high lift: 888


Support = 5 
Confidence = 40 
Rules = 4547
Rules with high lift: 735


Support = 5 
Confidence = 45 
Rules = 3826
Rules with high lift: 645


Support = 5 
Confidence = 50 
Rules = 3181
Rules with high lift: 570


Support = 5 
Confidence = 55 
Rules = 2476
Rules with high lift: 495


Support = 5 
Confidence = 60 
Rules = 1647
Rules with high lift: 427


Support = 5 
Confidence = 65 
Rules = 1216
Rules with high lift: 353


Support = 5 
Confidence = 70 
Rules = 934
Rules with high lift: 265


In [139]:
#HERE WE CAN SEE THE RULES THAT WE WANT
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=70, 
                report='YXlC') 
print('Number of rules:', len(rules))
#Code that filters all the association rules with a certain lift under
c = 0
for r in rules:
        if r[4]>1: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
            c += 1
            print('%s -- %s' % (str(c), r))
print('Rules with high lift: {}'.format(c))

Number of rules: 48
1 -- ('0_IsBadBuy', ('3_VehicleAge',), 87.65396145610278, 21.7558886509636, 1.0465275264313818, 91.73228346456693)
2 -- ('0_IsBadBuy', ('[60271.0, 71360.2)_VehOdo_bin',), 87.65396145610278, 22.25781584582441, 1.0164301319318185, 89.09412760717309)
3 -- ('0_IsBadBuy', ('4_VehicleAge',), 87.65396145610278, 23.44154175588865, 1.0164601103547068, 89.09675533469745)
4 -- ('0_IsBadBuy', ('CHEVROLET_Make',), 87.65396145610278, 23.70877944325482, 1.0290728523878725, 90.20231213872833)
5 -- ('0_IsBadBuy', ('OTHER_Auction',), 87.65396145610278, 23.97087794432548, 1.005183775160608, 88.10833988422783)
6 -- ('0_IsBadBuy', ('[1165.6, 1869.2)_WarrantyCost_bin', 'MANHEIM_Auction'), 87.65396145610278, 20.51220556745182, 1.0043174555789158, 88.03240354100551)
7 -- ('0_IsBadBuy', ('[7297.8, 10946.2)_VehBCost_bin', 'Alloy_WheelType'), 87.65396145610278, 22.836830835117773, 1.0241203927971636, 89.76820943665142)
8 -- ('0_IsBadBuy', ('[7297.8, 10946.2)_VehBCost_bin', 'MANHEIM_Auction'),

In [140]:
# count how many rows for each attribute to make sure i haven't cut anything
for i in df_new:
    conta = 0
    for j in df_new.iterrows():
        conta += 1
    print(i, " = ", conta)

Auction  =  58375
VehicleAge  =  58375
Make  =  58375
Trim  =  58375
Color  =  58375
WheelType  =  58375
Size  =  58375
VNST  =  58375
IsBadBuy  =  58375
VehBCost_bin  =  58375
VehOdo_bin  =  58375
WarrantyCost_bin  =  58375


In [141]:
df_new.IsBadBuy.value_counts()

0_IsBadBuy    51168
1_IsBadBuy     7207
Name: IsBadBuy, dtype: int64